<a href="https://colab.research.google.com/github/ArindamSarma/CSE-234-HW/blob/master/CSE_243_HW_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def ext_euclidean(a, b):

    t = u = 1
    s = v = 0
    while b:
        a, (q, b) = b, divmod(a, b)
        u, s = s, u - q * s
        v, t = t, v - q * t
    return a, u, v


class Point:

    def __init__(self, x, y):

        self.x = x
        self.y = y

    def __str__(self):

        return '(' + str(self.x) + ', ' + str(self.y) + ')'

    def __eq__(self, p):

        if type(self) != type(p):
            return False
        if self.x == p.x and self.y == p.y:
            return True
        return False


class EllipticCurve:

    def __init__(self, a, b, m):

        self.a = a
        self.b = b
        self.m = m

    def add(self, p, q):
        if p == None:
            return q
        if q == None:
            return p
        numerator = (q.y - p.y) % self.m
        denominator = (q.x - p.x) % self.m
        if denominator == 0:
            if p == q:
                if p.y == 0:
                    return None
                inv = ext_euclidean(2 * p.y, self.m)[1]
                slope = inv * (3 * p.x ** 2 + self.a) % self.m
            else:
                return None  
        else:
            inv = ext_euclidean(denominator, self.m)[1]
            slope = inv * numerator % self.m
        xr = (slope ** 2 - (p.x + q.x)) % self.m
        yr = (slope * (p.x - xr) - p.y) % self.m
        return Point(xr, yr)

    def mul(self, p, n):
        r = None
        for b in bin(n)[2:]:
            r = self.add(r, r)
            if b == '1':
                r = self.add(r, p)
        return r







In [25]:
p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
n = 6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449
a = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057148 % p
b = 1093849038073734274511112390766805569936207598951683748994586394495953116150735016013708737573759623248592132296706313309438452531591012912142327488478985984 % p
Gx = 2661740802050217063228768716723360960729859168756973147706671368418802944996427808491545080627771902352094241225065558662157113545570916814161637315895999846 % p
Gy = 3757180025770020463545507224491183603594455134769762486694567779615544477440556316691234405012945539562144444537289428522585666729196580810124344277578376784 % p 


ec = EllipticCurve(a, b, p)

gen = Point(Gx, Gy)

alice_private = 814709178348331822963098404943044035246972495353080501869149056740241 
bob_private = 1016189342726403936529228449371555925007815563308599801179551629341341

alice_public = ec.mul(gen, alice_private)
print("alice public key:", alice_public)

bob_public = B = ec.mul(gen, bob_private)
print("bob public key:", bob_public) 

alice_shared_key = ec.mul(bob_public, alice_private).x
print("alice shared key:", alice_shared_key)

bob_shared_key = ec.mul(alice_public, bob_private).x
print("bob shared key:", bob_shared_key)

sum_public = ec.add(alice_public, bob_public)
print("sum of public keys", sum_public)

alice public key: (1044303082331288333042920371742208774368155616296279065786423243070042275332732136363264652882200217061336900529157064346771811607632904560265846182493697624, 6730016909271769053897907753290829330809425963797605769853369810434278775560759851736336603039323983790910905138401345628507206516496489668105028448323383009)
bob public key: (5718419403644844610833599874967853512665697031913371072578001578978167810972857352497035598620709458817659568769905951977392840839865125603374463180503684656, 5146090798345568530884039262355125849229685323300901896331000538531014502583072954851119177218634029732278856706003599116733600206934060218548319836624316850)
alice shared key: 5554501709641722286443533888120161079362589870806449361292055404302521258459655288746670448272978103536337006842945731281500164474201110940499186364905930645
bob shared key: 5554501709641722286443533888120161079362589870806449361292055404302521258459655288746670448272978103536337006842945731281500164474201110

Used "Square roots from 1; 24, 51, 10 to Dan Shanks" by Ezra Brown as a reference:  https://www.maa.org/sites/default/files/pdf/upload_library/22/Polya/07468342.di020786.02p0470a.pdf, as well as https://en.wikipedia.org/wiki/Tonelli%E2%80%93Shanks_algorithm


In [0]:
def legendre_symbol(a, p):
    return pow(a, (p - 1) // 2, p)
  
def modInverse(a, m): 
    a = a % m; 
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1
 
def tonelli_shanks(n, p):
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1
    if s == 1:
        return pow(n, (p + 1) // 4, p)
    for z in range(2, p):
        if p - 1 == legendre_symbol(z, p):
            break
    c = pow(z, q, p)
    r = pow(n, (q + 1) // 2, p)
    t = pow(n, q, p)
    m = s
    t2 = 0
    while (t - 1) % p != 0:
        t2 = (t * t) % p
        for i in range(1, m):
            if (t2 - 1) % p == 0:
                break
            t2 = (t2 * t2) % p
        b = pow(c, 1 << (m - i - 1), p)
        r = (r * b) % p
        c = (b * b) % p
        t = (t * c) % p
        m = i
    return r

In [21]:
p = 407426149192123516349916653686361706730690466158799531798401
n = 5
r = tonelli_shanks(n, p)
print(r)
test_r_squared = pow(r,2,p)
print(test_r_squared)
neg_r = (-1*r)%p
print(neg_r)
test_neg_r_squared = pow(neg_r,2,p)
print(test_neg_r_squared)

403534680041342292531656861460378171833866469000243831072020
5
3891469150781223818259792225983534896823997158555700726381
5
